In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sinkerDL = pd.read_csv('DraftLeagueSinker.csv')
sinkerMLB = pd.read_csv('MLBSinker.csv')

In [3]:
sinkerMLB = sinkerMLB[sinkerMLB['pitches'] > 50]
sinkerMLB.drop('pitches', inplace=True, axis=1)

In [4]:
sinkerDL['league'] = 'Draft League'
sinkerMLB['league'] = 'MLB'

In [5]:
sinkerDL = sinkerDL.astype(str)
sinkerMLB = sinkerMLB.astype(str)

In [6]:
pitchersDL = list(set(sinkerDL['pitcher']))

In [7]:
slider = pd.concat([sinkerDL, sinkerMLB])

In [8]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [9]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [10]:
def get_index_from_pitcher(pitcher):
    return sinker[sinker.pitcher == pitcher].index[0]

In [11]:
def get_pitcher_from_index(index):
    return sinker[sinker.index == index].values[0][0]

In [12]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        sinker = pd.concat([sinkerDL, sinkerMLB])

        sinker["combined_features"] = sinker.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(sinker["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        sinker = sinker[(sinker['pitcher'] == pitcher_like) | (sinker['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass

In [13]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp = cosine_comp.sort_values('MLB DL Pitcher')
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
71,"Alba, Miguel",No Comp,No Similarity
72,"Allain, Luke","Musgrove, Joe",0.3
61,"Amerlaan, Ashe","García, Edgar",0.435194
59,"Ammerlaan, Ashe",No Comp,No Similarity
7,"Ayers, Kyle",No Comp,No Similarity
...,...,...,...
21,"Wilhite, Thomas","Iglesias, Raisel",0.3
89,"Wilson, Benjamin","Garcia, Bryan",0.316228
69,"Wojtarowicz, Andre","Raley, Brooks",0.316228
25,"Wright, Kerry","García, Luis",0.435194


In [14]:
cosine_comp.to_csv('sinker_cosine_comp.csv', sep='\t')